Stelle alle Daten aus der Datenbank sowie das Maximalgewicht der beiden Transporter bereit.

In [1]:
import sqlite3
connection = sqlite3.connect("hardware.db")
cursor = connection.cursor()
cursor.execute("SELECT * FROM hardware;")
hardware = cursor.fetchall()
cursor.close()
connection.close()

maxweight = 1100000
drivers = [72400, 85700]

In [2]:
print(hardware)

[('Notebook Büro 13"', 205, 2451, 40), ('Notebook Büro 14"', 420, 2978, 35), ('Notebook outdoor', 450, 3625, 80), ('Mobiltelefon Büro', 60, 717, 30), ('Mobiltelefon Outdoor', 157, 988, 60), ('Mobiltelefon Heavy Duty', 220, 1220, 65), ('Tablet Büro klein', 620, 1405, 40), ('Tablet Büro groß', 250, 1455, 40), ('Tablet outdoor klein', 540, 1690, 45), ('Tablet outdoor groß', 370, 1980, 68)]


Hierbei handelt es sich um ein 0/1-Knapsack-Problem mit zwei Rucksäcken, das mit dynamischer Programmierung optimal gelöst werden kann. Bei Unterteilung in 1-Gramm-Schritte werden allerdings die zugehörigen Matrizen zu groß (ca. 1 Mio * 1 Mio * 3300 bzw. 1 Mio * 1 Mio * 2 bei platzsparendem Code). Eine separate Optimierung für beide Rucksäcke ist zwar möglich (zweimal ca. 1 Mio * 3300 bzw. 1 Mio * 2 bei platzsparendem Code), garantiert aber kein globales Optimum.
Daher wird zunächst ein Greedy-Algorithmus verwendet. Dazu wird für jedes Gerät ein "Profit" als Nutzwert/Gewicht berechnet und die Geräte dann nach Profit absteigend in die Transporter geladen. Ob zunächst alles in Transporter 1 und dann in Transporter 2 geladen wird oder beide Transporter abwechselnd beladen werden, macht in diesem konkreten Fall keinen Unterschied (auch bei der lokalen Optimierung später nicht), daher wird nur letztere Variante dargestellt.

Berechne zunächst die Profite und sortiere die Geräte danach.

In [3]:
profits = [i[3] / i[2] for i in hardware]
hardware = [i for _,i in sorted(zip(profits, hardware), reverse=True)]
print(hardware)

[('Mobiltelefon Outdoor', 157, 988, 60), ('Mobiltelefon Heavy Duty', 220, 1220, 65), ('Mobiltelefon Büro', 60, 717, 30), ('Tablet outdoor groß', 370, 1980, 68), ('Tablet Büro klein', 620, 1405, 40), ('Tablet Büro groß', 250, 1455, 40), ('Tablet outdoor klein', 540, 1690, 45), ('Notebook outdoor', 450, 3625, 80), ('Notebook Büro 13"', 205, 2451, 40), ('Notebook Büro 14"', 420, 2978, 35)]


Die Funktion findmax berechnet die optimale Beladung der Transporter nach dem Greedy-Algorithmus. Dazu wird die Hardware als Liste von Tupeln sowie die Kapazitäten der beiden Transporter übergeben. Das Ergebnis ist ein Dictionary mit den Namen der Geräte und der zugehörigen Anzahl.

In [4]:
def findmax(data, t1, t2):
    # Initialisiere Daten
    names = [i[0] for i in data]
    need = [i[1] for i in data]
    weights = [i[2] for i in data]
    values = [i[3] for i in data]
    # Erzeuge Transporter
    truck1 = {key:value for key, value in zip(names, [0 for i in range(len(names))])}
    truck2 = {key:value for key, value in zip(names, [0 for i in range(len(names))])}
    # Speichere, wie viel bereits verladen wurde
    used1 = 0
    used2 = 0
    # Belade abwechselnd die beiden Transporter nach Profit der Hardware
    for item in range(len(need)):
        while need[item] > 0 and (used1 + weights[item] <= t1 or used2 + weights[item] <= t2):
            if used1 + weights[item] <= t1 and used2 + weights[item] <= t2:
                if used1 <= used2:
                    truck1[names[item]] += 1
                    used1 = used1 + weights[item]
                else:
                    truck2[names[item]] += 1
                    used2 = used2 + weights[item]
                need[item] -= 1
            elif used1 + weights[item] <= t1:
                truck1[names[item]] += 1
                used1 = used1 + weights[item]
                need[item] -= 1
            elif used2 + weights[item] <= t2:
                truck2[names[item]] += 1
                used2 = used2 + weights[item]
                need[item] -= 1
            else:
                break
    return truck1, truck2

Berechne die Beladung der Transporter mithilfe des Greedy-Algorithmus.

In [5]:
truck1, truck2 = findmax(hardware, maxweight-drivers[0], maxweight-drivers[1])
print(truck1)
print(truck2)

{'Mobiltelefon Outdoor': 79, 'Mobiltelefon Heavy Duty': 110, 'Mobiltelefon Büro': 29, 'Tablet outdoor groß': 185, 'Tablet Büro klein': 304, 'Tablet Büro groß': 0, 'Tablet outdoor klein': 0, 'Notebook outdoor': 0, 'Notebook Büro 13"': 0, 'Notebook Büro 14"': 0}
{'Mobiltelefon Outdoor': 78, 'Mobiltelefon Heavy Duty': 110, 'Mobiltelefon Büro': 31, 'Tablet outdoor groß': 185, 'Tablet Büro klein': 295, 'Tablet Büro groß': 0, 'Tablet outdoor klein': 0, 'Notebook outdoor': 0, 'Notebook Büro 13"': 0, 'Notebook Büro 14"': 0}


Die Funktions totals berechnet das Gewicht der verladenen Hardware sowie die Nutzwerte der zwei Transporter.

In [6]:
def totals(data, t1, t2):
    gewicht1 = sum([t1[data[i][0]] * data[i][2] for i in range(len(data))])
    gewicht2 = sum([t2[data[i][0]] * data[i][2] for i in range(len(data))])
    wert1 = sum([t1[data[i][0]] * data[i][3] for i in range(len(data))])
    wert2 = sum([t2[data[i][0]] * data[i][3] for i in range(len(data))])
    print("Gewicht 1: " + str(gewicht1))
    print("Wert 1: " + str(wert1))
    print("Gewicht 2: " + str(gewicht2))
    print("Wert 2: " + str(wert2))
    print("Gewicht 1+2: " + str(gewicht1 + gewicht2))
    print("Wert 1+2: " + str(wert1 + wert2))

Berechne die verladenen Gewichte und Nutzwerte je Transporter.

In [7]:
totals(hardware, truck1, truck2)

Gewicht 1: 1026465
Wert 1: 37500
Gewicht 2: 1014266
Wert 2: 37140
Gewicht 1+2: 2040731
Wert 1+2: 74640


Lokale Optimierung: Im folgenden Schritt wird in der Nähe der Greedy-Lösung eine bessere Lösung gesucht. Dazu werden 25 Geräte je Typ wieder ausgepackt und mit dynamischer Programmierung der verbleibende Platz im Transporter optimal genutzt (die 25 ist so gewählt, dass das Programm zumindest auf meinem PC in vertretbarer Zeit läuft). Da noch Geräte der Art 'Tablet Büro klein' übrig sind, werden dabei 'Notebook Büro 13', 'Notebook Büro 14' und 'Tablet Büro groß' nicht berücksichtigt, da sie höchstens den gleichen Nutzwert wie 'Tablet Büro klein' haben, aber mehr wiegen und es daher immer besser ist, 'Tablet Büro klein' einzupacken.

In [8]:
local = [i for i in hardware if i[0] not in ('Notebook Büro 13"', 'Notebook Büro 14"', 'Tablet Büro groß')]
print(local)

[('Mobiltelefon Outdoor', 157, 988, 60), ('Mobiltelefon Heavy Duty', 220, 1220, 65), ('Mobiltelefon Büro', 60, 717, 30), ('Tablet outdoor groß', 370, 1980, 68), ('Tablet Büro klein', 620, 1405, 40), ('Tablet outdoor klein', 540, 1690, 45), ('Notebook outdoor', 450, 3625, 80)]


In [9]:
import numpy as np

# Anzahl der ausgepackten Geräte je Typ
unpack = 25

# Initialisiere Daten
need1 = [0 for i in range(len(local))]
need2 = [0 for i in range(len(local))]

total1 = 0
actual1 = need1
total2 = 0
actual2 = need2

t1 = findmax(local, maxweight-drivers[0], maxweight-drivers[1])[0]
t2 = findmax(local, maxweight-drivers[0], maxweight-drivers[1])[1]

rest1 = maxweight-drivers[0]  - sum([max(0,(t1[local[i][0]] - unpack)) * local[i][2] for i in range(len(local)-2)])
rest2 = maxweight-drivers[1]  - sum([max(0,(t2[local[i][0]] - unpack)) * local[i][2] for i in range(len(local)-2)])

weights = np.repeat([i[2] for i in local], unpack)
values = np.repeat([i[3] for i in local], unpack)

size = weights.size
mat1 = np.zeros((size,rest1+1), dtype=int)
keep1 = np.zeros((size,rest1+1), dtype=int)
mat2 = np.zeros((size,rest2+1), dtype=int)
keep2 = np.zeros((size,rest2+1), dtype=int)

# Berechnung der optimalen Verteilung mit dynamischer Programmierung
for i in range(size):
    for j in range(rest1+1):
        if weights[i] <= j:
            if max(mat1[i-1][j], values[i] + mat1[i-1][j-weights[i]]) == mat1[i-1][j]:
                mat1[i][j] = mat1[i-1][j]
            else:
                keep1[i][j] = 1
                mat1[i][j] = values[i] + mat1[i-1][j-weights[i]]
        else:
            mat1[i][j] = mat1[i-1][j]
    for k in range(rest2+1):
        if weights[i] <= k:
            if max(mat2[i-1][k], values[i] + mat2[i-1][k-weights[i]]) == mat2[i-1][k]:
                mat2[i][k] = mat2[i-1][k]
            else:
                keep2[i][k] = 1
                mat2[i][k] = values[i] + mat2[i-1][k-weights[i]]
        else:
            mat2[i][k] = mat2[i-1][k]

# Ausgabe der optimalen Verteilung
selected1 = np.array([], dtype=int)
i, j = size-1, rest1
while(i >= 0):
    if keep1[i][j] == 1:
        selected1 = np.append(selected1, i)
        j = j-weights[i]
    i = i - 1

selected2 = np.array([], dtype=int)
i, j = size-1, rest2
while(i >= 0):
    if keep2[i][j] == 1:
        selected2 = np.append(selected2, i)
        j = j-weights[i]
    i = i - 1

names = [i[0] for i in local]
truck1rest = {key:value for key, value in zip(names, [0 for i in range(len(names))])}
truck2rest = {key:value for key, value in zip(names, [0 for i in range(len(names))])}

for i in selected1:
    truck1rest[np.repeat(names, unpack)[i]] += 1
print(truck1rest)
for i in selected2:
    truck2rest[np.repeat(names, unpack)[i]] += 1
print(truck2rest)

{'Mobiltelefon Outdoor': 25, 'Mobiltelefon Heavy Duty': 25, 'Mobiltelefon Büro': 24, 'Tablet outdoor groß': 25, 'Tablet Büro klein': 25, 'Tablet outdoor klein': 1, 'Notebook outdoor': 0}
{'Mobiltelefon Outdoor': 25, 'Mobiltelefon Heavy Duty': 25, 'Mobiltelefon Büro': 25, 'Tablet outdoor groß': 25, 'Tablet Büro klein': 25, 'Tablet outdoor klein': 0, 'Notebook outdoor': 0}


Die lokale Optimierung zeigt, dass statt eines 'Mobiltelefon Büro' besser ein 'Tablet outdoor klein' eingepackt werden sollte.

In [10]:
truck1['Tablet outdoor klein'] += 1
truck1['Mobiltelefon Büro'] -= 1
print(truck1)
print(truck2)
totals(hardware, truck1, truck2)

{'Mobiltelefon Outdoor': 79, 'Mobiltelefon Heavy Duty': 110, 'Mobiltelefon Büro': 28, 'Tablet outdoor groß': 185, 'Tablet Büro klein': 304, 'Tablet Büro groß': 0, 'Tablet outdoor klein': 1, 'Notebook outdoor': 0, 'Notebook Büro 13"': 0, 'Notebook Büro 14"': 0}
{'Mobiltelefon Outdoor': 78, 'Mobiltelefon Heavy Duty': 110, 'Mobiltelefon Büro': 31, 'Tablet outdoor groß': 185, 'Tablet Büro klein': 295, 'Tablet Büro groß': 0, 'Tablet outdoor klein': 0, 'Notebook outdoor': 0, 'Notebook Büro 13"': 0, 'Notebook Büro 14"': 0}
Gewicht 1: 1027438
Wert 1: 37515
Gewicht 2: 1014266
Wert 2: 37140
Gewicht 1+2: 2041704
Wert 1+2: 74655
